In [ ]:
import os
# import ipympl
from pathlib import Path
from pdb import set_trace

from datetime import datetime
import numpy as np #For general mathematical calculations
import matplotlib.pyplot as plt #For plotting
from scipy.integrate import trapz
from scipy.signal import savgol_filter #Allows to smooth a profile through the Savitzky-Golay filter
import xarray as xr
#For working with lidar products (developed by the Atmospheric Physics Group of the UGR, GFAT)
from gfatpy.atmo import ecmwf, atmo

from gfatpy.lidar.reader import reader_xarray
from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import date_quicklook
from gfatpy.lidar.quality_assurance.rayleigh_fit import rayleigh_fit_from_date
from gfatpy.lidar import utils
from gfatpy.lidar.retrieval import klett
from gfatpy.lidar.retrieval.helper import RetrievalHelper
from gfatpy.lidar.preprocessing.lidar_preprocessing_tools import average_dc_signal, subtract_dark_current, subtract_background
from gfatpy.aeronet.reader import reader as aeronet_reader
from gfatpy.aeronet.utils import find_aod

from gfatpy.utils.plot import font_axes, color_list
from gfatpy.atmo import solar

import lidar_preprocessing

%load_ext autoreload
%autoreload 2

%matplotlib inline
# %matplotlib widget

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
databasepath = Path(r'G:\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_elastico\data\1a')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
databasepath = Path(r'C:\Users\Usuario\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_elastico\data\1a')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
outputpath =   Path(r'C:\Users\Usuario\Downloads\geomet_output')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

databasepath.is_dir(), outputpath.is_dir() #Comprueba que ambas carpetas existen.

In [ ]:
filelist = databasepath /'2021'/'06'/'24'/'*Prs*.nc'
date_ini = "2021-06-24"
channels = ["532xpa", '532xsa']
ini_date = '20210624T223000'   #Initial date and time. Format 'yyyymmddThhmmss', where T represents literally the letter T. Example: '20200101T200000'
end_date = '20210624T233000'   #Same format than ini_date

#Lidar measurement
lidar = reader_xarray(filelist, date_ini=ini_date, date_end=end_date, channels=channels)

#Dark measurement
dc_fl = databasepath / '2021' / '06' / '24' / '*Pdc*.nc'
dc_ds = reader_xarray(dc_fl, channels=channels)

colors = {'532xpa': 'darkgreen', '532xsa': 'lime'}
fig,ax = plt.subplots(figsize=[10,6])
lidar.signal_532xpa.mean('time').plot(x='range',ax=ax,color=colors['532xpa'], label='532xpa-raw')
dc_ds.signal_532xpa.mean('time').plot(x='range',ax=ax,color=colors['532xpa'], ls='dashed', label='532xpa-DC')
lidar.signal_532xsa.mean('time').plot(x='range',ax=ax,color=colors['532xsa'], label='532xsa-raw')
dc_ds.signal_532xsa.mean('time').plot(x='range',ax=ax,color=colors['532xsa'], ls='dashed', label='532xsa-DC')
ax.set_yscale('log')
ax.set_xlim(0,15000)
ax.set_ylim(7,10)
ax.legend(fontsize=14)

In [ ]:
channels = ["532xpa", '532xsa', '532xsp', '532xpp']
lidar = preprocess(filelist, 
dc_fl=dc_fl,
ini_date= ini_date,
end_date= end_date,
channels=channels)

In [ ]:
lidar['signal_532xsa'].mean('time').plot(x='range')
lidar['signal_532xpa'].mean('time').plot(x='range')
plt.ylim(-0.025,0.01)

In [ ]:
datestr, hour_ini, hour_end = '20210624', '223000', '233000' 
date_ini = datetime.strptime(datestr,'%Y%m%d')
date_ = date_ini
year, month, day = date_.year, date_.month, date_.day
ini_date_str = '%sT%s' % (datestr, hour_ini)
end_date_str = '%sT%s' % (datestr, hour_end)
rcs_532xpa = lidar['signal_532xpa']*lidar.range**2
rcs_532xpp = lidar['signal_532xpp']*lidar.range**2
rcs_532xsa = lidar['signal_532xsa']*lidar.range**2
rcs_532xsp = lidar['signal_532xsp']*lidar.range**2
rcs_532xpa_profile = rcs_532xpa.sel(time=slice(ini_date_str,end_date_str)).mean('time')
rcs_532xpp_profile = rcs_532xpp.sel(time=slice(ini_date_str,end_date_str)).mean('time')
rcs_532xsa_profile = rcs_532xsa.sel(time=slice(ini_date_str,end_date_str)).mean('time')
rcs_532xsp_profile = rcs_532xsp.sel(time=slice(ini_date_str,end_date_str)).mean('time')
nrcs_532xpa = rcs_532xpa_profile/rcs_532xpa_profile.sel(range=slice(8000,9000)).mean('range')
nrcs_532xpp = rcs_532xpp_profile/rcs_532xpp_profile.sel(range=slice(8000,9000)).mean('range')
nrcs_532xsa = rcs_532xsa_profile/np.abs(rcs_532xsa_profile.sel(range=slice(8000,9000)).mean('range'))
nrcs_532xsp = rcs_532xsp_profile/rcs_532xsp_profile.sel(range=slice(8000,9000)).mean('range')


In [ ]:
ranges = lidar.range
wavelength = 532
meteo_profiles = ecmwf.get_ecmwf_temperature_pressure(
    datestr, ranges_array=ranges
)


# Molecular Attenuated Backscatter
mol_properties = atmo.molecular_properties(
    wavelength, meteo_profiles['pressure'], meteo_profiles['temperature'], ranges
)
beta_mol_att = mol_properties["attenuated_molecular_beta"]


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
nrcs_532xpa.plot(x='range', label='532xpa', ax=ax)
nrcs_532xpp.plot(x='range', label='532xpp', lw=4, ax=ax)
nrcs_532xsa.plot(x='range', label='532xsa', ax=ax)
nrcs_532xsp.plot(x='range', label='532xsp',lw=1, ax=ax)
(beta_mol_att/beta_mol_att.sel(range=slice(8000,9000)).mean('range')).plot(x='range',label='mol',color='k', ax=ax)
ax.legend()
ax.set_yscale('log')
ax.set_ylim(0.1,10)
# plt.savefig('rayleigh_gfatpy.png')